In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
for dirname,_,filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

./data\personality_dataset.csv
./data\sample_submission.csv
./data\test.csv
./data\train.csv


In [4]:
df=pd.read_csv('./data/train.csv')
df_test=pd.read_csv('./data/test.csv')
df_sample_submission=pd.read_csv('./data/sample_submission.csv')

In [5]:
df.shape

(18524, 9)

In [6]:
df['Personality'].value_counts()

Personality
Extrovert    13699
Introvert     4825
Name: count, dtype: int64

In [7]:
df.head(5)

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert


In [8]:
print(df['Stage_fear'].unique())
print(df['Drained_after_socializing'].unique())

['No' 'Yes' nan]
['No' nan 'Yes']


In [10]:
mapping = {'Yes': 1, 'No': 0}
df['Stage_fear'] = df['Stage_fear'].map(mapping)

mapping1 = {'Yes': 1, 'No': 0}
df['Drained_after_socializing'] = df['Drained_after_socializing'].map(mapping1)

mapping2 = {'Extrovert': 1, 'Introvert': 0}
df['Personality'] = df['Personality'].map(mapping2)

In [11]:
df.head(5)

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,NaN,6.0,4.0,0.0,15.0,5.0,1
1,1,1.0,NaN,7.0,3.0,0.0,10.0,8.0,1
2,2,6.0,NaN,1.0,0.0,NaN,3.0,0.0,0
3,3,3.0,NaN,7.0,3.0,0.0,11.0,5.0,1
4,4,1.0,NaN,4.0,4.0,0.0,13.0,NaN,1


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18524 entries, 0 to 18523
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         18524 non-null  int64  
 1   Time_spent_Alone           17334 non-null  float64
 2   Stage_fear                 0 non-null      float64
 3   Social_event_attendance    17344 non-null  float64
 4   Going_outside              17058 non-null  float64
 5   Drained_after_socializing  17375 non-null  float64
 6   Friends_circle_size        17470 non-null  float64
 7   Post_frequency             17260 non-null  float64
 8   Personality                18524 non-null  int64  
dtypes: float64(7), int64(2)
memory usage: 1.3 MB


In [13]:
df.isnull().sum()

id                               0
Time_spent_Alone              1190
Stage_fear                   18524
Social_event_attendance       1180
Going_outside                 1466
Drained_after_socializing     1149
Friends_circle_size           1054
Post_frequency                1264
Personality                      0
dtype: int64